In [1]:
import pandas as pd
import numpy as np
import re as re
import requests
from bs4 import BeautifulSoup
from pymongo import MongoClient
import folium
import googlemaps
from datetime import datetime
import os
from dotenv import load_dotenv
load_dotenv()

def getPage(url):
    res = requests.get(url)
    html = res.text
    soup = BeautifulSoup(html, 'html.parser')
    return soup

## Web scraping para encontrar locales en alquiler en Amsterdam

In [2]:
url = "http://www.dutchrental.com/commercial-property/Amsterdam"
soup = getPage(url)
rlocal = soup.select('p a')

In [3]:
web =[]
for r in rlocal:
    web.append(r.get('href'))
print(web)

['/commercial-property-for-rent/Amsterdam/Corsicaweg-10/770329/overview', '/commercial-property-for-rent/Amsterdam/Robert-Kochplantsoen-19/770331/overview', '/commercial-property-for-rent/Amsterdam/Eerste-Leliedwarsstraat-6/744638/overview', '/commercial-property-for-rent/Amsterdam/Geuzenstraat-98/744754/overview', '/commercial-property-for-rent/Amsterdam/Jan-Rebelstraat-18-C/739903/overview', '/commercial-property-for-rent/Amsterdam/Willebroekstraat-62/733702/overview', '/commercial-property-for-rent/Amsterdam/Admiraal-de-Ruijterweg-402/650655/overview', '/commercial-property-for-rent/Amsterdam/Geuzenstraat-98/644697/overview']


In [4]:
web1=[]
for i in range (2,6):
    x = getPage(f"http://www.dutchrental.com/commercial-property/Amsterdam?page={i}")   
    rlocal1 = x.select('p a')
    for r in rlocal1:
        web1.append(r.get('href'))
print(web1)


['/commercial-property-for-rent/Amsterdam/Nieuw-Zeelandweg-6-E/571247/overview', '/commercial-property-for-rent/Amsterdam/Meentstraat-66/558209/overview', '/commercial-property-for-rent/Amsterdam/Johan-Huizingalaan-110/558606/overview', '/commercial-property-for-rent/Amsterdam/Keurenplein-17-A/515379/overview', '/commercial-property-for-rent/Amsterdam/Socratesstraat-26/466064/overview', '/commercial-property-for-rent/Amsterdam/Stadhouderskade-55-parkeerplaats/462061/overview', '/commercial-property-for-rent/Amsterdam/Rhoneweg-18/449573/overview', '/commercial-property-for-rent/Amsterdam/Stadhouderskade-55/408508/overview', '/commercial-property-for-rent/Amsterdam/Petroleumhavenweg-22/360434/overview', '/commercial-property-for-rent/Amsterdam/Licentiehouder-gezocht/358640/overview', '/commercial-property-for-rent/Amsterdam/Willemsparkweg-135-B/346519/overview', '/commercial-property-for-rent/Amsterdam/Contactweg-131/204353/overview', '/commercial-property-for-rent/Amsterdam/Hemonystraat

In [5]:
import re
cleaning = web+web1
clean =[]
for c in cleaning:
    c1 = re.sub("/commercial-property-for-rent/Amsterdam/", "", c)
    c2 = re.sub("\/[0-9]+\/overview", "", c1)
    c3 = re.sub("\-", " ", c2)
    clean.append(c3)
clean

['Corsicaweg 10',
 'Robert Kochplantsoen 19',
 'Eerste Leliedwarsstraat 6',
 'Geuzenstraat 98',
 'Jan Rebelstraat 18 C',
 'Willebroekstraat 62',
 'Admiraal de Ruijterweg 402',
 'Geuzenstraat 98',
 'Nieuw Zeelandweg 6 E',
 'Meentstraat 66',
 'Johan Huizingalaan 110',
 'Keurenplein 17 A',
 'Socratesstraat 26',
 'Stadhouderskade 55 parkeerplaats',
 'Rhoneweg 18',
 'Stadhouderskade 55',
 'Petroleumhavenweg 22',
 'Licentiehouder gezocht',
 'Willemsparkweg 135 B',
 'Contactweg 131',
 'Hemonystraat 66',
 'Nieuwe Hemweg 4 c',
 'Veemarkt 212 F',
 'Siriusdreef 17 27']

In [6]:
gmaps = googlemaps.Client(key=os.getenv("key"))


## Con las direcciones saco las coordenadas

In [7]:
def coordLocals(listlocals):
    addresslocals = []
    for n in listlocals:
        addresslocals.append(gmaps.places(n))      
    return addresslocals

localams = coordLocals(clean)

In [8]:
localams[0]['results'][0]['geometry']['location']['lng']

4.8115896

## Saco los parametros que quiero de los locales

In [9]:
def getParameters(companies):
    param =[]
    for comp in companies:
        for i in range(len(comp['results'])):
            longitude = comp['results'][i]['geometry']['location']['lng']
            latitude = comp['results'][i]['geometry']['location']['lat']
            address = comp['results'][i]['name']
            loc = {
            'address': address,
            'coordinates':[longitude, latitude]
            }
            param.append(loc)
    return param

In [10]:
x = getParameters(localams)

## Lo subo a mongodb y creo la localización para el geoindex

In [11]:
def connectCollection(database, collection):
    client = MongoClient()
    db = client[database]
    coll = db[collection]
    return db, coll

In [12]:
db, coll = connectCollection('companies', 'companies')

In [13]:
col = db["rentlocals"]
col.insert_many(x)

In [14]:
def getLocation(companies):
    location =[]
    longitude = companies['coordinates'][1]
    latitude = companies['coordinates'][0]
    loc = {
        'type':'Point',
        'coordinates':[latitude, longitude]
    }
    location.append(loc)
    print(location)
    return location

In [15]:
x[0]

{'address': 'Corsicaweg 10',
 'coordinates': [4.8115896, 52.4021632],
 '_id': ObjectId('5dd7cc20ead1fc712e30354a')}

In [16]:
db, collr = connectCollection('companies', 'rentlocals')


In [ ]:
for x1 in x:
    value = {"$set": {'location':getLocation(x1)}}
    query = {"_id": x1['_id']}
    collr.update_many(query, value)

In [18]:
%store x

Stored 'x' (list)


In [ ]:
# Con esto compruebo si puedo hacer web scraping con selenium la primera o de manera normal como la segunda, 
# crea un archivo en la carpeta que estoy y si se abre en el navegador es que puedo hacerlo sin selenium

test = requests.get("https://www.fundainbusiness.nl/en/alle-bedrijfsaanbod/amsterdam/").text
with open("test.html","w+") as file:
    file.write(test)

    
test = requests.get("http://www.dutchrental.com/commercial-property/Amsterdam").text
with open("test.html","w+") as file:
    file.write(test)